In [1]:
import os
import sys
PROJECT_PATH = "/home/albert/Baikal-ML/" #insert your project path
sys.path.append(f"{PROJECT_PATH}")

In [3]:
import torch
from nnetworks.layers.config import LstmConfig
from nnetworks.layers.recurrent import LstmLayer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x = torch.rand((3,10,5))
mask = torch.ones((3,10,1))
mask[:,:,8:] = 0.

print(mask.shape)

print()

layer1 = LstmLayer(LstmConfig(5, 32, return_sequences=True, dropout=0)).to(device)
layer2 = LstmLayer(LstmConfig(64, 32, return_sequences=False, dropout=0)).to(device)

x, mask = layer1(x.to(device), mask.to(device))
x, mask = layer2(x, mask)
print(x.shape)
print(x)

torch.Size([3, 10, 1])

torch.Size([3, 64])
tensor([[ 0.8007, -0.6698,  0.2658, -0.4245, -1.4589, -0.3294,  0.5927, -1.0696,
          0.8649, -0.4737, -1.2497,  0.8164, -0.7533,  0.9898,  0.0628,  0.1957,
          1.2311,  0.6330,  2.1823,  1.7677,  1.3194,  1.5399, -0.3620, -0.6027,
         -1.0231, -2.2173, -1.2308, -0.0849, -0.4654, -0.5583, -0.2949,  0.6827,
         -0.3444, -0.6856, -0.3484, -0.5654, -0.4156,  0.9231, -0.2047,  1.8161,
         -1.0772, -0.6705, -0.1179,  0.9374, -0.3811,  1.1696, -0.0111, -0.5084,
         -0.0367, -0.7520, -0.7550,  1.9995,  1.9023,  0.7913, -0.5466, -1.7673,
          0.5004, -0.2736,  1.1735, -0.5016, -1.2207, -1.9663,  1.1416,  0.1189],
        [ 0.7934, -0.6662,  0.2671, -0.4218, -1.4534, -0.3160,  0.6020, -1.0786,
          0.8640, -0.4732, -1.2472,  0.8289, -0.7506,  0.9863,  0.0669,  0.2017,
          1.2211,  0.6344,  2.1872,  1.7647,  1.3138,  1.5336, -0.3602, -0.5902,
         -1.0331, -2.1978, -1.2298, -0.0878, -0.4663, -0.5648, -

In [5]:
torch.cuda.device_count()

2

In [ ]:
seq_lengths = mask.permute((0,2,1)).sum((1,2)).int()
x_packed = torch.nn.utils.rnn.pack_padded_sequence(x.permute((0,2,1)), seq_lengths, batch_first=True, enforce_sorted=False)
x_unpacked, lens = torch.nn.utils.rnn.pad_packed_sequence(x_packed, batch_first=True)

In [ ]:
x_unpacked

tensor([[[0.9602, 0.7778, 0.9035, 0.5809, 0.8522],
         [0.9988, 0.7547, 0.6862, 0.9939, 0.3699],
         [0.1989, 0.0195, 0.1859, 0.2255, 0.0515],
         [0.0825, 0.5559, 0.7703, 0.8230, 0.5491],
         [0.2470, 0.5166, 0.9346, 0.2490, 0.8540],
         [0.9943, 0.3321, 0.4404, 0.0339, 0.7577],
         [0.8129, 0.0427, 0.8189, 0.8150, 0.7795],
         [0.1300, 0.4416, 0.6594, 0.3438, 0.2170]],

        [[0.5098, 0.4356, 0.2161, 0.6960, 0.7278],
         [0.9826, 0.3290, 0.8029, 0.5891, 0.0460],
         [0.3145, 0.2974, 0.3226, 0.3008, 0.1838],
         [0.7262, 0.7762, 0.5743, 0.4647, 0.2241],
         [0.3346, 0.7425, 0.2410, 0.4885, 0.9742],
         [0.1619, 0.6273, 0.3818, 0.4822, 0.4060],
         [0.3273, 0.9638, 0.8088, 0.6849, 0.5540],
         [0.9408, 0.5814, 0.1703, 0.5672, 0.3454]],

        [[0.0629, 0.0971, 0.6380, 0.0012, 0.1572],
         [0.7093, 0.2241, 0.9507, 0.7804, 0.0749],
         [0.6821, 0.0795, 0.1010, 0.5428, 0.0317],
         [0.6596, 0.2987, 0

In [ ]:
lens

tensor([8, 8, 8])

In [ ]:
lstm = torch.nn.LSTM(
            input_size=5,
            hidden_size=32,
            num_layers=1,
            batch_first=True,
            dropout=0,
            bidirectional=True # merge mode is concat by torch realization
        )

In [ ]:
x_out, _ = lstm(x_packed)

In [ ]:
x_out

PackedSequence(data=tensor([[-8.9833e-02,  1.4103e-02,  6.9557e-02,  ..., -3.7554e-02,
         -8.4614e-05,  1.8708e-01],
        [-6.1768e-02,  3.2840e-02,  5.3113e-02,  ..., -3.4246e-02,
          1.7363e-02,  1.6012e-01],
        [-6.7546e-02, -7.2765e-03,  8.2122e-02,  ..., -3.9059e-02,
          1.6766e-02,  1.6971e-01],
        ...,
        [-1.7902e-01,  1.1500e-02,  1.3763e-01,  ...,  1.2252e-02,
         -1.2653e-02,  6.0942e-02],
        [-1.5605e-01,  3.4304e-02,  1.2970e-01,  ..., -1.7229e-02,
          2.9652e-02,  6.5387e-02],
        [-1.6116e-01,  1.0137e-02,  1.4287e-01,  ..., -2.4414e-02,
          3.8958e-02,  9.5758e-02]], grad_fn=<CatBackward0>), batch_sizes=tensor([3, 3, 3, 3, 3, 3, 3, 3]), sorted_indices=tensor([0, 1, 2]), unsorted_indices=tensor([0, 1, 2]))